In [1]:
import pandas as pd

In [2]:
from address_match_proto import is_valid_street_simplea, ret_options
#from address_match_proto import ret_options


In [3]:
global ZIP_DF

In [4]:

file_path = '../2_develop_synthetic_company/ZIP_Locale_Detail.xls'

# Read the Excel file into a Pandas DataFrame
ZIP_DF = pd.read_excel(file_path, dtype=str)

# Display the DataFrame
ZIP_DF.head()

,AREA NAME,AREA CODE,DISTRICT NAME,DISTRICT NO,DELIVERY ZIPCODE,LOCALE NAME,PHYSICAL DELV ADDR,PHYSICAL CITY,PHYSICAL STATE,PHYSICAL ZIP,PHYSICAL ZIP 4
0,SOUTHERN,4G,PUERTO RICO,006,00601,ADJUNTAS,37 CALLE MUNOZ RIVERA,ADJUNTAS,PR,00601,9998
1,SOUTHERN,4G,PUERTO RICO,006,00602,AGUADA,5 AVE NATIVO ALERS,AGUADA,PR,00602,9998
2,SOUTHERN,4G,PUERTO RICO,006,00603,AGUADILLA,50 CARR 459 STE 1,AGUADILLA,PR,00603,9998
3,SOUTHERN,4G,PUERTO RICO,006,00604,RAMEY,100 AVE BORINQUEN,AGUADILLA,PR,00603,9996
4,SOUTHERN,4G,PUERTO RICO,006,00605,AGUADILLA,50 CARR 459 STE 1,AGUADILLA,PR,00603,9998


In [5]:
ZIP_DF.columns

Index(['AREA NAME', 'AREA CODE', 'DISTRICT NAME', 'DISTRICT NO',
       'DELIVERY ZIPCODE', 'LOCALE NAME', 'PHYSICAL DELV ADDR',
       'PHYSICAL CITY', 'PHYSICAL STATE', 'PHYSICAL ZIP', 'PHYSICAL ZIP 4'],
      dtype='object')

In [6]:
zip2_df = ZIP_DF[['PHYSICAL CITY', 'PHYSICAL STATE', 'PHYSICAL ZIP']]

In [7]:
zip2_df.head()

,PHYSICAL CITY,PHYSICAL STATE,PHYSICAL ZIP
0,ADJUNTAS,PR,00601
1,AGUADA,PR,00602
2,AGUADILLA,PR,00603
3,AGUADILLA,PR,00603
4,AGUADILLA,PR,00603


In [8]:
vdf=pd.read_csv('synthetic_two_company.csv')

Index(['AREA NAME', 'AREA CODE', 'DISTRICT NAME', 'DISTRICT NO',
       'DELIVERY ZIPCODE', 'LOCALE NAME', 'PHYSICAL DELV ADDR',
       'PHYSICAL CITY', 'PHYSICAL STATE', 'PHYSICAL ZIP', 'PHYSICAL ZIP 4'],
      dtype='object')

In [9]:
class ZIP:

    def is_valid(self, zip_code):
        global ZIP_DF
        filtered_rows = ZIP_DF[ZIP_DF['DELIVERY ZIPCODE'] == zip_code]
        if filtered_rows.empty:
            return False
        else:
            return True
    
    def is_equal(self, zip_code, city):
        global ZIP_DF
        filtered_rows = ZIP_DF[ZIP_DF['DELIVERY ZIPCODE'] == zip_code]
        #print(filtered_rows['LOCALE NAME'].values[0])
        locale_name = filtered_rows['LOCALE NAME'].values[0]
        physical_city = filtered_rows['PHYSICAL CITY'].values[0]
        # print(locale_name, physical_city)
        if city.upper() == locale_name:
            return True

        if city.upper() == physical_city:
            return True
        else:
            return False
   
    def is_5_digits(self, zip_code):
        if len(zip_code) == 5:
            return True
        else:
            return False




In [10]:
def qb_validate_zip(df_row):
    #print(df_row.Zip, df_row.City)
    # Example usage:
    zip_instance = ZIP()
     # Test is_equal method
    qb_zip = str(df_row.Zip)
    qb_city = df_row.City

    zip_valid = zip_instance.is_valid(qb_zip)
    if not zip_valid:
        print(f"zip_valid result: {zip_valid}")
        return False
    # Test is_5_digits method
    digits_result = zip_instance.is_5_digits(qb_zip)
    #print(f"is_5_digits result: {digits_result}")

    if not digits_result:
        print(f"is_5_digits result: {digits_result}")
        return False

    
    zip_result = zip_instance.is_equal(qb_zip, qb_city)
    #print(f"is_equal result: {zip_result}")

    if not zip_result:
        print(f"City and ZIP is_equal result: {zip_result}")
        return False

    return True

In [11]:
def find_longest_word(input_string):
    # Split the input string into words
    words = input_string.split()

    # Find the longest word
    longest_word = max(words, key=len)

    return longest_word



In [12]:
def longest_street_word(df_o, qb_street_address, qb_city, qb_state, qb_zip):
    long_word = find_longest_word(qb_street_address)
    print('LONG:',long_word)
    df_lw = df_o[df_o['street'].str.contains(long_word, case=False, na=False)]
    return df_lw
    

In [13]:
def is_valid_street_complex(qb_street_address, qb_city, qb_state, qb_zip):
    print('COMPLEX', qb_street_address, qb_city, qb_state, qb_zip)
    df_o = ret_options(qb_street_address, qb_city, qb_state, qb_zip)
    num = df_o.shape[0]
    print(f'it could be any one of {num} options')
    print(df_o.head(2))
    df_l = longest_street_word(df_o, qb_street_address, qb_city, qb_state, qb_zip)
    num1 = df_l.shape[0]
    print(f'it could be any one of {num1} LONG WORD options')
    print(df_l.head(2))
    if num1 == 1:
        return True
    else:
        return False


In [14]:
def qb_validate_street(df_row):
    qb_zip = str(df_row.Zip)
    qb_street_address = df_row['Street Address']
    qb_state = df_row.State
    qb_city = df_row.City

    #print(qb_street_address, qb_city, qb_state, qb_zip)
    a_status = is_valid_street_simplea(qb_street_address, qb_city, qb_state, qb_zip)
    if not a_status:
        a_status = is_valid_street_complex(qb_street_address, qb_city, qb_state, qb_zip)
    print(a_status)
    return a_status
    

In [15]:
bad_list_of_dicts = []
for i, my_df_row in vdf.iterrows():
    #print(type(my_df_row))
    #print(my_df_row)
    if qb_validate_zip(my_df_row):
        pass
    else:
        print('BIG PROBLEM')
        print(my_df_row)
        print('-'*30)
        bad_list_of_dicts.append(my_df_row.to_dict())
        continue
    
    if qb_validate_street(my_df_row):
        pass
    else:
        print('SMALL PROBLEM')
        #print(my_df_row)
        print('-'*30)
        bad_list_of_dicts.append(my_df_row.to_dict())

True
COMPLEX 219 N BROADWAY ST AURORA SD 57002
it could be any one of 10 options
            id zipcode number          street street2    city state  \
257  190942610   57002    219  N BROADWAY AVE          AURORA    SD   
258  190926251   57002  21903       476TH AVE          AURORA    SD   

      latitude  longitude  
257  44.284573 -96.687947  
258  44.209724 -96.686968  
LONG: BROADWAY
it could be any one of 1 LONG WORD options
            id zipcode number          street street2    city state  \
257  190942610   57002    219  N BROADWAY AVE          AURORA    SD   

      latitude  longitude  
257  44.284573 -96.687947  
True
COMPLEX 132 ST OLAF AVE BALTIC SD 57003
it could be any one of 1 options
           id zipcode number          street street2    city state   latitude  \
51  191113201   57003    132  SAINT OLAF AVE          BALTIC    SD  43.760436   

    longitude  
51 -96.740052  
LONG: OLAF
it could be any one of 1 LONG WORD options
           id zipcode number         

In [16]:
bad_df = pd.DataFrame(bad_list_of_dicts)

In [17]:
bad_df

,Vendor,Company,Street Address,City,State,Country,Zip,1099 Tracking,Phone,Email,Attachments,Open Balance
0,Happy,NaN,410 4TH ST,CHESTER,SD,NaN,57016,NaN,NaN,NaN,NaN,0.0
1,Mahoney Mugs,Mahoney Mugs,112 MAIN ST,CROOKS,SD,NaN,57020,NaN,NaN,NaN,NaN,0.0
2,Met Life Dental,Met Life Dental,110 E US HIGHWAY 18,DAVIS,SD,NaN,57021,NaN,NaN,NaN,NaN,0.0
3,National Eye Care,National Eye Care,305 E 4TH ST,DELL RAPIDS,SD,NaN,57022,NaN,(800) 555-5555,"Nateyecare@intuit.com, pauliejones15@intuit.com",NaN,0.0
4,Robertson & Associates,Robertson & Associates,320 LINCOLN AVE,FAIRVIEW,SD,NaN,57027,NaN,(650) 557-1111,NaN,NaN,315.0
5,Tony Rondonuwu,NaN,506 W 25TH ST,YANKTON,SD,NaN,57078,NaN,NaN,tonyrjr@intuit.com,NaN,0.0


In [18]:
my_df_row.to_dict()

{'Vendor': 'United States Treasury',
 'Company': nan,
 'Street Address': '518 N CLIFF AVE',
 'City': 'Sioux Falls',
 'State': 'SD',
 'Country': nan,
 'Zip': 57103,
 '1099 Tracking': nan,
 'Phone': '(888) 555-6454',
 'Email': 'taxesaregreat@intuit.com',
 'Attachments': nan,
 'Open Balance': 0.0}

In [19]:
bad_df.to_csv('outliers.csv')